In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/metacognitive-dataset/metacognitive-dataset.csv
/kaggle/input/modified-dataset/modified_dataset.csv
/kaggle/input/metacognitive-feedback-for-algorithm-solving/final_dataset_with_annotated_metacognitive_feedback_gpt-4o-mini.csv


In [4]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModel, AutoTokenizer, GPT2Model,GPT2Tokenizer,GPT2LMHeadModel
from transformers import AutoTokenizer, T5ForConditionalGeneration , T5Tokenizer , T5Model
import pandas as pd

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cuda')

In [8]:
checkpoint = "t5-base"

In [9]:
t5_tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
t5_tokenizer.vocab_size

32100

In [11]:
#set the max length to model's default present max length
t5_tokenizer.model_max_length = t5_tokenizer.model_max_length

In [12]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [13]:
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

In [14]:
file_path = "/kaggle/input/metacognitive-feedback-for-algorithm-solving/final_dataset_with_annotated_metacognitive_feedback_gpt-4o-mini.csv"
df = pd.read_csv(file_path)

In [15]:
df.columns

Index(['Question 1', 'Response 1', 'Right answer 1', 'Q01', 'Q02', 'Q03',
       'Q04', 'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13',
       'Q14', 'Q15', 'Q16', 'metacognitive_vector', 'metacognitive_feedback'],
      dtype='object')

In [16]:
df.rename(
    columns={
        'Question 1': 'Problem',
        'Response 1': 'Student_code',
        'Right answer 1': 'Expected_code'
    },
    inplace=True
)

In [17]:
df.head(3)

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
0,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...","Your initial code serves as a starting point, ..."
1,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",Your code exhibits a solid attempt at reading ...
2,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",It looks like you're in a good place with some...


In [18]:
df['combined_problem_student'] = df['Problem'] + " " + df['Student_code']

In [19]:
df['combined_problem_expected'] = df['Problem'] + " " + df['Expected_code']

In [20]:
df.columns

Index(['Problem', 'Student_code', 'Expected_code', 'Q01', 'Q02', 'Q03', 'Q04',
       'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14',
       'Q15', 'Q16', 'metacognitive_vector', 'metacognitive_feedback',
       'combined_problem_student', 'combined_problem_expected'],
      dtype='object')

In [21]:
df.dropna(subset=['Problem', 'metacognitive_feedback', 'combined_problem_student'], inplace=True)

In [22]:
df.reset_index(drop=True, inplace=True)

In [23]:
df.isnull().sum()

Problem                      0
Student_code                 0
Expected_code                0
Q01                          0
Q02                          0
Q03                          0
Q04                          0
Q05                          0
Q06                          0
Q07                          0
Q08                          0
Q09                          0
Q10                          0
Q11                          0
Q12                          0
Q13                          0
Q14                          0
Q15                          0
Q16                          0
metacognitive_vector         0
metacognitive_feedback       0
combined_problem_student     0
combined_problem_expected    0
dtype: int64

In [24]:
df['metacognitive_feedback'][100]

"Your current implementation shows a good effort in structuring your code with functions, but there are some areas where further refinement is necessary to meet the problem requirements. First, consider how you're capturing the relationships between birth years and heights. While you are correctly gathering names, birthdates, and heights into a dictionary, think about how you can aggregate heights by decade instead of year. This will streamline the calculation of average heights. It’s crucial to loop through your input list once to comprehend and process the data before beginning any calculations for averaging—possibly consolidating this logic in your `calculate_average_height` function. Also, ensure that you are converting heights to the correct data type before performing any arithmetic operations. Additionally, take a closer look at how you’re determining the range of decades; right now it seems like you may be focusing on unique years instead of decades. Consider creating a systema

In [25]:
df.head(5)

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback,combined_problem_student,combined_problem_expected
0,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...","Your initial code serves as a starting point, ...",Develop a Python program that takes the name o...,Develop a Python program that takes the name o...
1,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",Your code exhibits a solid attempt at reading ...,Develop a Python program that takes the name o...,Develop a Python program that takes the name o...
2,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",It looks like you're in a good place with some...,Develop a Python program that takes the name o...,Develop a Python program that takes the name o...
3,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",Your approach to reading the file and splittin...,Develop a Python program that takes the name o...,Develop a Python program that takes the name o...
4,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",Your initial approach to the problem is a good...,Develop a Python program that takes the name o...,Develop a Python program that takes the name o...


In [26]:
from torch.utils.data import Dataset
import ast
class CustomDataset(Dataset):
    def __init__(self, dataset, t5_tokenizer,gpt2_tokenizer, max_length=512):
        self.t5_tokenizer = t5_tokenizer
        self.gpt2_tokenizer = gpt2_tokenizer
        self.data = dataset
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        metacognitive_vector = self.data['metacognitive_vector'][idx]
        problem_student_code = self.data['combined_problem_student'][idx]
        problem_expected_code = self.data['combined_problem_expected'][idx]
        student_code = self.data['Student_code'][idx]
        target = self.data['metacognitive_feedback'][idx]

        metacognitive_vector_float = [
        float(item.strip()) for item in ast.literal_eval(metacognitive_vector)]
        metacognition_vector_ids = torch.tensor(metacognitive_vector_float, dtype=torch.float)
        
        problem_student_code_ids = torch.tensor(
            self.t5_tokenizer.encode(problem_student_code, max_length=self.max_length, truncation=True, padding="max_length")
        )
        problem_expected_code_ids = torch.tensor(
            self.t5_tokenizer.encode(problem_expected_code, max_length=self.max_length, truncation=True, padding="max_length")
        )
        
        student_code_ids = torch.tensor(
            self.t5_tokenizer.encode(student_code, max_length=self.max_length, truncation=True, padding="max_length")
        )
        target_ids = torch.tensor(
            self.t5_tokenizer.encode(target, max_length=self.max_length, truncation=True, padding="max_length")
        )

        return metacognition_vector_ids, problem_student_code_ids, problem_expected_code_ids, student_code_ids, target_ids

In [27]:
dataset = CustomDataset(df, t5_tokenizer, gpt2_tokenizer)

In [28]:
len(dataset)

16803

In [29]:
metacognition_vector_ids, problem_student_code_ids, problem_expected_code_ids, student_code_ids, target_ids = dataset[100]
print(f"Metacognition vector IDs: {metacognition_vector_ids}")
print(f"Expected feedback IDs: {problem_student_code_ids.shape}")
print(f"Expected encoded feedback IDs: {problem_expected_code_ids.shape}")
print(f"Student Answer IDs: {student_code_ids}")
print(f"Target IDs: {target_ids}")
print("\n")

Metacognition vector IDs: tensor([2., 3., 1., 3., 3., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 2.])
Expected feedback IDs: torch.Size([512])
Expected encoded feedback IDs: torch.Size([512])
Student Answer IDs: tensor([ 3785,   834, 11966,  4350,  2423,    77,  2562,  9960,  1713,   188,
         2176,    15,     6,  2122, 31911,  2294,  2394,     6,  2938, 27436,
         1713,   279,    32,   115,     6,  4928, 20223, 13523,  4433,     6,
        27640,     5,   357,  1713, 18610,   760,   630,     6,  2884, 31497,
         2294,  3072,     6,  2938, 19419,  1713,   308,     9,  6961,     6,
         1714,    87,  4305, 13523,  3301,     6,  2606, 16029,  1713,   427,
          162,     6,  2517,    87,  5176, 13523,  2079,     6,  2517, 12100,
           20,    89,   608,   834, 11966,   599,    77,  2562,   834, 11966,
         4350,    61,    10,    28,   539,   599,    77,  2562,   834, 11966,
         4350,   976,    52,  8512,    38,  1042,    10, 10223,  2423, 11966,
           

In [30]:
gpt2_tokenizer.pad_token

'<|endoftext|>'

In [31]:
gpt2_pad_token_id = gpt2_tokenizer.pad_token_id

In [32]:
gpt2_pad_token_id

50256

In [33]:
t5_tokenizer.pad_token

'<pad>'

In [34]:
t5_pad_token_id = t5_tokenizer.pad_token_id

In [35]:
t5_pad_token_id

0

# Context Enocder

In [72]:
class ContextEncoder(nn.Module): 
    def __init__(self, t5_model_name='t5-base', output_dim=768): 
        super(ContextEncoder, self).__init__()
        
        self.t5_encoder = T5Model.from_pretrained(t5_model_name).encoder
        self.fc = nn.Linear(self.t5_encoder.config.d_model, output_dim)
    
    def forward(self, problem_student_code_ids, attention_masks=None):        

        encoder_outputs = self.t5_encoder(
            input_ids=problem_student_code_ids,
            attention_mask=attention_masks
        )
        context_hidden_states = encoder_outputs.last_hidden_state   
        
        context_rep = context_hidden_states.mean(dim=1)
        
       
        context_rep = self.fc(context_rep)
        final_rep = context_rep.unsqueeze(1)
        
        return final_rep

In [73]:
context_encoder = ContextEncoder().to(device)

In [74]:
context_encoder

ContextEncoder(
  (t5_encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout(p=0.1, inplace=False)
              (a

In [75]:
class MetacognitionLayer(nn.Module):
    def __init__(self, metacognitive_dim=16, output_dim=768):
        super(MetacognitionLayer, self).__init__()
        #16 to 768 mapping
        self.metacognitive_fc = nn.Linear(metacognitive_dim, output_dim) 
        self.final_fc = nn.Linear(output_dim, output_dim)

    def forward(self, metacognitive_vector):
        

        metacognitive_rep = self.metacognitive_fc(metacognitive_vector)
        final_rep = self.final_fc(metacognitive_rep)  
        persona_rep = final_rep.unsqueeze(1)

        return persona_rep

In [76]:
metacognitive_emb = MetacognitionLayer().to(device)

In [42]:
metacognitive_emb

MetacognitionLayer(
  (metacognitive_fc): Linear(in_features=16, out_features=1024, bias=True)
  (final_fc): Linear(in_features=1024, out_features=1024, bias=True)
)

In [43]:
class PAALayer(nn.Module):
    def __init__(self, hidden_dimension = 768 , tau=0.8,dropout_rate=0.1):
        super(PAALayer, self).__init__()
        self.hidden_dimenstion = hidden_dimension
        self.tau = tau

       
        self.fc = nn.Linear(2 * hidden_dimension, hidden_dimension)  
        self.sigmoid = nn.Sigmoid()
        self.fc_out = nn.Linear(hidden_dimension, hidden_dimension)
        self.dropout = nn.Dropout(p=dropout_rate)


    def forward(self, hR , oP, oC):
       
        Mp_input  = torch.cat([hR,oP], dim=-1)        
        Mp = self.fc(Mp_input)      
        Wp = self.sigmoid(Mp)      
     
        Mpersona = Wp
        Mcontext = 1 - Wp      
       
        oP_weighted = Mpersona * oP       
        oC_weighted = Mcontext * oC
       
        HPAA = oP_weighted + oC_weighted 
       
        output = self.fc_out(HPAA)
       
        return output

In [44]:
paa = PAALayer()

In [45]:
paa

PAALayer(
  (fc): Linear(in_features=1536, out_features=768, bias=True)
  (sigmoid): Sigmoid()
  (fc_out): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [87]:
class CustomTransformerBlock(nn.Module):
    def __init__(self, hidden_size, tau , dropout_rate=0.1):
        super(CustomTransformerBlock, self).__init__()

        self.input_self_attention = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)


        self.context_attn = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)
        self.persona_attn = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)
        #self.output_attn = nn.MultiheadAttention(hidden_size, num_head = 12, batch_first = True)

        #self.output_self_attention = nn.MultiheadAttention(hidden_size, num_heads=12 , batch_first = True)

        self.paa_layer = PAALayer(hidden_dimension=hidden_size, tau=tau)
        
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(2048, hidden_size),
            nn.LayerNorm(hidden_size)
        )
        self.layer_norm2 = nn.LayerNorm(hidden_size)
       
    def forward(self, current_state, encoded_persona, encoded_context):

        hR, _ = self.input_self_attention(current_state, current_state, current_state)
        oP, _ = self.persona_attn(hR, encoded_persona, encoded_persona)
        oC, _ = self.context_attn(oP, encoded_context, encoded_context)   

        
        HPAA = self.paa_layer(hR, oP, oC)
        
        mlp_output = self.mlp(PC)
        output = self.layer_norm2(mlp_output + PC)
      
        return output


In [88]:
custom_layer = CustomTransformerBlock(768,0.5)

In [89]:
custom_layer

CustomTransformerBlock(
  (input_self_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (context_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (persona_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (output_self_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (paa_layer): PAALayer(
    (fc): Linear(in_features=1536, out_features=768, bias=True)
    (sigmoid): Sigmoid()
    (fc_out): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (mlp): Sequential(
    (0): Linear(in_features=768, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=2048, out_features

In [90]:
class PAAModel(nn.Module):
    def __init__(self, hidden_size=768, vocab_size = 32100 ,tau=0.8, max_length=512, num_transformer_blocks=4):
        super(PAAModel , self).__init__()
        self.hidden_size = hidden_size
        self.tau = tau
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.num_transformer_blocks = num_transformer_blocks

        self.context_encoder = ContextEncoder()
        
        self.metacognitive_emb = MetacognitionLayer()
        
        for param in self.context_encoder.parameters():
            param.requires_grad = False

        for param in self.metacognitive_emb.parameters():
            param.requires_grad = False

        self.token_embedding = nn.Embedding(vocab_size, hidden_size)
        self.position_embedding = nn.Embedding(max_length, hidden_size)
        self.dropout = nn.Dropout(p=0.1) 

        # self.self_attention = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)        
        self.transformer_blocks = nn.ModuleList([CustomTransformerBlock(hidden_size, tau) for _ in range(num_transformer_blocks)])
        self.final_fc = nn.Linear(hidden_size, vocab_size)
        self.layer_norm = nn.LayerNorm(hidden_size)

    def forward(self,  metacognitive_vector_ids,
                       problem_student_code_ids ,
                       problem_expected_code_ids ,
                       expected_attention_mask,
                       student_attention_mask):

        metacognitive_vector_emb = self.metacognitive_emb(metacognitive_vector_ids)

        problem_student_encoded = self.context_encoder(problem_student_code_ids,
                                                attention_masks=student_attention_mask)
        problem_expected_encoded = self.context_encoder(problem_expected_code_ids,
                                                       attention_masks = expected_attention_mask)
        
        seq_length = problem_student_encoded.size(1)
        token_embeds = self.token_embedding(problem_student_code_ids)
        position_ids = torch.arange(0, seq_length, device=problem_student_code_ids.device).unsqueeze(0)
        position_embeds = self.position_embedding(position_ids)
        
        # Combine embeddings
        inputs_embeds = token_embeds + position_embeds
        inputs_embeds = self.dropout(inputs_embeds)
      
        current_state = inputs_embeds
        for transformer_block in self.transformer_blocks:
            transformer_output = transformer_block(current_state, metacognitive_vector_emb, problem_expected_encoded)           
       
        logits = self.final_fc(transformer_output)    

        return logits

In [91]:
model = PAAModel()

In [92]:
model.to(device)

PAAModel(
  (context_encoder): ContextEncoder(
    (t5_encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_features=3072, out_features=768, bias=False)


In [52]:
torch.cuda.empty_cache()

In [53]:
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [54]:
LOSS = torch.nn.CrossEntropyLoss()

In [55]:
num_epochs = 200

In [56]:
df_train = df[0:5000]

In [57]:
len(df_train)

5000

In [58]:
df_train.isnull().sum()

Problem                      0
Student_code                 0
Expected_code                0
Q01                          0
Q02                          0
Q03                          0
Q04                          0
Q05                          0
Q06                          0
Q07                          0
Q08                          0
Q09                          0
Q10                          0
Q11                          0
Q12                          0
Q13                          0
Q14                          0
Q15                          0
Q16                          0
metacognitive_vector         0
metacognitive_feedback       0
combined_problem_student     0
combined_problem_expected    0
dtype: int64

In [59]:
from torch.utils.data import DataLoader
train_dataset = CustomDataset(df_train, t5_tokenizer , gpt2_tokenizer)
train_dataloader = DataLoader(train_dataset , batch_size = 1 ,shuffle = True )

In [60]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

KeyboardInterrupt: 

In [61]:
checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
for epoch in range(num_epochs):
    print(f"Training started for epoch {epoch + 1}/{num_epochs}")
    model.train()
    total_loss = 0

    for idx, (metacognition_vector_ids,
              problem_student_code_ids,
              problem_expected_code_ids,
              student_code_ids,
              target_ids) in enumerate(train_dataloader):
        
        metacognition_vector_ids = metacognition_vector_ids.to(device)
        problem_student_code_ids = problem_student_code_ids.to(device)
        problem_expected_code_ids = problem_expected_code_ids.to(device)
        student_code_ids = student_code_ids.to(device)
        target_ids = target_ids.to(device)
        
        #attention masking
        student_attention_mask = (problem_student_code_ids != t5_pad_token_id).long().to(device)
        expected_attention_mask = (problem_expected_code_ids != t5_pad_token_id).long().to(device)
        

        optimizer.zero_grad()
        logits = model(metacognition_vector_ids,
                       problem_student_code_ids ,
                       problem_expected_code_ids ,
                       expected_attention_mask,
                       student_attention_mask)

     
        logits = logits.view(-1, logits.size(-1))
        target_ids = target_ids.view(-1)

       
        loss = LOSS(logits, target_ids)
        total_loss += loss.item()

      
        loss.backward()       
        for name, param in model.named_parameters():
            if 'context_encoder' in name:
                assert param.grad is None, f"Gradients found in frozen encoder {name}"
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)       
        optimizer.step()    
        
        if idx % 10 == 0:
            print(f"Batch {idx + 1}/{len(train_dataloader)} | Loss: {loss.item():.4f}" , end='\r')




    
                    
    if epoch % 10 ==0 :
            for name, param in model.named_parameters():
                if param.requires_grad and param.grad is not None:
                    print(f"Layer: {name} | Grad Norm: {param.grad.norm().item()}")
                elif param.requires_grad:
                    print(f"Layer: {name} | Grad: None")       
    
    if (epoch + 1) % 20 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch + 1}.pth")
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss / max(len(train_dataloader), 1),
        }, checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")

   
    avg_loss = total_loss / max(len(train_dataloader), 1)
    #writer.add_scalar("Loss/train", avg_loss, epoch + 1)
    print(f"Epoch [{epoch + 1}/{num_epochs}] completed | Average Loss: {avg_loss:.4f}")

Training started for epoch 1/200
Layer: token_embedding.weight | Grad Norm: 0.0
Layer: position_embedding.weight | Grad Norm: 0.0
Layer: transformer_blocks.0.input_self_attention.in_proj_weight | Grad: None
Layer: transformer_blocks.0.input_self_attention.in_proj_bias | Grad: None
Layer: transformer_blocks.0.input_self_attention.out_proj.weight | Grad: None
Layer: transformer_blocks.0.input_self_attention.out_proj.bias | Grad: None
Layer: transformer_blocks.0.context_attn.in_proj_weight | Grad: None
Layer: transformer_blocks.0.context_attn.in_proj_bias | Grad: None
Layer: transformer_blocks.0.context_attn.out_proj.weight | Grad: None
Layer: transformer_blocks.0.context_attn.out_proj.bias | Grad: None
Layer: transformer_blocks.0.persona_attn.in_proj_weight | Grad: None
Layer: transformer_blocks.0.persona_attn.in_proj_bias | Grad: None
Layer: transformer_blocks.0.persona_attn.out_proj.weight | Grad: None
Layer: transformer_blocks.0.persona_attn.out_proj.bias | Grad: None
Layer: transform

In [1]:
torch.save(model.state_dict(), 'paamodel.pth')

NameError: name 'torch' is not defined

In [ ]:
model = PAAModel()

# Load the saved state dict
model.load_state_dict(torch.load('paamodel.pth'))

In [ ]:
df_eval = df[9999:10000].reset_index(drop=True)

In [ ]:
df_eval

In [ ]:
eval_dataset = CustomDataset(df_eval, t5_tokenizer,gpt2_tokenizer)

In [ ]:


def inference(model,gpt2_tokenizer, t5_tokenizer, eval_dataset, device):
    model.eval()
    model.to(device) 

    metacognitive_vector_ids, problem_student_code_ids, problem_expected_code_ids,student_code_ids, target_ids = eval_dataset[0]

    metacognitive_tensor = metacognitive_vector_ids.unsqueeze(0).to(device)  
    problem_student_code_tensor = problem_student_code_ids.unsqueeze(0).to(device)
    problem_expected_code_tensor = problem_expected_code_ids.unsqueeze(0).to(device)
    target_tensor = target_ids.unsqueeze(0).to(device)

    student_attention_mask = (problem_student_code_tensor != t5_tokenizer.pad_token_id).long().to(device)
    context_attention_mask = (problem_expected_code_tensor != t5_tokenizer.pad_token_id).long().to(device)

    
    with torch.no_grad():         
        
        logits = model(
            metacognitive_vector_ids=metacognitive_tensor,
            problem_student_code_ids=problem_student_code_tensor,
            problem_expected_code_ids=problem_expected_code_tensor,
            context_attention_mask=context_attention_mask,
            student_attention_mask=student_attention_mask
        )
        
        predictions = logits.argmax(dim=-1).squeeze().tolist()  
        decoded_text = t5_tokenizer.decode(predictions, skip_special_tokens=True)

        
        return predictions, decoded_text



In [ ]:
predictions, decoded_text = inference(model, gpt2_tokenizer, t5_tokenizer, eval_dataset, device)

In [ ]:
print("Predicted Tokens:", predictions)

In [ ]:
print("Decoded Text:", decoded_text)

In [ ]:
def load_checkpoint(checkpoint_path, model, optimizer=None):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded: Epoch {epoch}, Loss: {loss:.4f}")
    return model, optimizer, epoch, loss

In [ ]:
checkpoint_path = "./checkpoints/model_epoch_89.pth"
model, optimizer, start_epoch, _ = load_checkpoint(checkpoint_path, model, optimizer)

In [ ]:
checkpoint_path = "./checkpoints/model_epoch_10.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))  # Use GPU if available: 'cuda'
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
len(df)

In [ ]:
df_eval1 = df[449:450].reset_index(drop=True)

In [ ]:
eval_dataset1 = CustomDataset(df_eval1, t5_tokenizer,gpt2_tokenizer)

In [ ]:
predictions, decoded_text = inference(model, gpt2_tokenizer, t5_tokenizer, eval_dataset1, device)

In [ ]:
print("Decoded Text:", decoded_text)